# NOAA Weather API Exploration

Exploring the weather.gov API to understand data structures for building a Data Vault model.

**Goal:** Map ski resorts to forecast zones and observation stations, then design a data pipeline.

In [ ]:
import requests
import json

# Required User-Agent header for weather.gov API
headers = {
    'User-Agent': '(portfolio-weather-app, nate@example.com)',
    'Accept': 'application/geo+json'
}

# Test location: Sunday River, ME
test_lat = 44.4667
test_lon = -70.8500

## 1. Points Endpoint - Get Grid/Zone Info for a Location

This is the starting point - converts lat/lon to grid coordinates, zones, and forecast URLs.

In [ ]:
response = requests.get(
    f"https://api.weather.gov/points/{test_lat},{test_lon}",
    headers=headers
)

print(f"GET /points/{test_lat},{test_lon}")
print(f"Status: {response.status_code}\n")
print(json.dumps(response.json(), indent=2))

## 2. Forecast Zones - List All Zones for a State

Get all forecast zones in Maine. Each zone covers a geographic area.

In [ ]:
response = requests.get(
    "https://api.weather.gov/zones",
    headers=headers,
    params={'area': 'ME', 'type': 'forecast'}
)

print("GET /zones?area=ME&type=forecast")
print(f"Status: {response.status_code}\n")
print(json.dumps(response.json()["features"][0], indent=2))

## 3. Observation Stations - List All Stations for a State

Get all observation stations in Maine that provide current conditions.

In [ ]:
response = requests.get(
    "https://api.weather.gov/stations",
    headers=headers,
    params={'state': 'ME'}
)

print("GET /stations?state=ME")
print(f"Status: {response.status_code}\n")
print(json.dumps(response.json()["features"][0], indent=2))
# print(response.json()["features"])

## 4. Grid Forecast (12-hour periods)

Standard 7-day forecast with 12-hour periods (day/night).

In [ ]:
# Get grid coordinates from points endpoint first
points_response = requests.get(
    f"https://api.weather.gov/points/{test_lat},{test_lon}",
    headers=headers
)
forecast_url = points_response.json()['properties']['forecast']

# Get the forecast
response = requests.get(forecast_url, headers=headers)

print(f"GET {forecast_url}")
print(f"Status: {response.status_code}\n")
print(json.dumps(response.json(), indent=2))

## 5. Hourly Forecast

Hour-by-hour forecast for the next 7 days.

In [ ]:
# Get hourly forecast URL from points endpoint
points_response = requests.get(
    f"https://api.weather.gov/points/{test_lat},{test_lon}",
    headers=headers
)
forecast_hourly_url = points_response.json()['properties']['forecastHourly']

# Get the hourly forecast
response = requests.get(forecast_hourly_url, headers=headers)

print(f"GET {forecast_hourly_url}")
print(f"Status: {response.status_code}\n")
print(json.dumps(response.json(), indent=2))

## 6. Grid Data (Raw Forecast)

Raw numerical forecast data - temperature, wind, precipitation, etc.

In [ ]:
# Get grid data URL from points endpoint
points_response = requests.get(
    f"https://api.weather.gov/points/{test_lat},{test_lon}",
    headers=headers
)
forecast_grid_url = points_response.json()['properties']['forecastGridData']

# Get the grid data
response = requests.get(forecast_grid_url, headers=headers)

print(f"GET {forecast_grid_url}")
print(f"Status: {response.status_code}\n")
print(json.dumps(response.json(), indent=2))

## 7. Station Observations (Current Conditions)

Latest observation from a weather station.

In [ ]:
# Get nearest stations from points endpoint
points_response = requests.get(
    f"https://api.weather.gov/points/{test_lat},{test_lon}",
    headers=headers
)
stations_url = points_response.json()['properties']['observationStations']

# Get list of stations
stations_response = requests.get(stations_url, headers=headers)
station_id = stations_response.json()['features'][0]['properties']['stationIdentifier']

# Get latest observation from first station
response = requests.get(
    f"https://api.weather.gov/stations/{station_id}/observations/latest",
    headers=headers
)

print(f"GET /stations/{station_id}/observations/latest")
print(f"Status: {response.status_code}\n")
print(json.dumps(response.json(), indent=2))

## 8. Zone Forecast

Forecast for an entire zone (alternative to grid-based forecast).

In [ ]:
# Get zone ID from points endpoint
points_response = requests.get(
    f"https://api.weather.gov/points/{test_lat},{test_lon}",
    headers=headers
)
zone_url = points_response.json()['properties']['forecastZone']
zone_id = zone_url.split('/')[-1]

# Get zone forecast
response = requests.get(
    f"https://api.weather.gov/zones/forecast/{zone_id}/forecast",
    headers=headers
)

print(f"GET /zones/forecast/{zone_id}/forecast")
print(f"Status: {response.status_code}\n")
print(json.dumps(response.json(), indent=2))